In [1]:
from bs4 import BeautifulSoup
from multiprocessing.pool import ThreadPool
import numpy as np
import pandas as pd
import make_data
import datetime
import requests
import glob
import sys
import os
import json
import time
import re

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestCentroid, KNeighborsClassifier, NearestNeighbors
from sklearn.cross_validation import cross_val_score
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.linear_model import RidgeClassifier, LinearRegression, LogisticRegression

/home/adil/.pyenv/versions/3.6.1/envs/python3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
# names = [91, 111, 106, 104, 101, 327, 331, 332, 335, 94, 346, 348, 352, 401, 798, 92, 245, 325, 97, 1198, 1124, 1126, 1205, 1206, 1183, 1204, 1186, 1187, 1188, 1190, 1191, 90, 147, 159, 161, 1080, 157, 1081, 1100, 1101, 1102, 1103, 150, 173, 174, 175, 443, 1099, 154, 176, 177, 178, 179, 180, 156, 158, 162, 163, 1104, 1237, 164, 165, 166, 167, 916, 919, 168, 169, 170, 171, 172, 917, 918, 1105, 89, 141, 142, 143, 139, 136, 137, 138, 133, 135, 134, 131, 126, 130, 129, 127, 128, 123, 125, 124, 116, 122, 119, 121, 120, 117, 118, 936, 105, 114, 115, 113, 112, 110, 108, 107, 937, 103, 93, 98, 102, 99, 100, 96, 95, 132, 144, 145, 146, 148, 149, 151, 152, 153, 457, 458, 637, 638, 459, 460, 461, 639, 640, 462, 464, 641, 466, 595, 596, 597, 636, 181, 182, 193, 200, 446, 197, 218, 219, 220, 221, 222, 223, 224, 225, 198, 201, 235, 202, 240, 243, 448, 449, 451, 452, 203, 204, 445, 970, 226, 228, 229, 230, 231, 227, 199, 249, 250, 263, 274, 276, 277, 1388, 1422, 292, 1389, 303, 306, 307, 308, 1384, 1385, 1386, 1387, 310, 312, 314, 316, 1424, 1145, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 456, 419, 420, 421, 422, 838, 839, 840, 841, 842, 843, 884, 423, 424, 425, 426, 427, 1497, 428, 429, 430, 997, 1028, 1029, 1030, 1031, 1046, 1144, 1121, 1127, 1261, 1262, 1334, 1430, 431, 1189, 432, 433, 435, 436, 437, 439, 442, 453, 454, 1079, 705, 194, 196, 210, 1499, 1500, 1501, 706, 707, 708, 709, 711, 725, 712, 713, 714, 715, 716, 717, 767, 768, 1143, 1214, 1345, 720, 721, 722, 1213, 723, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 492, 493, 494, 495, 496, 497, 498, 499, 500, 501, 502, 503, 504, 505, 506, 507, 560, 563, 564, 566, 570, 589, 1074, 576, 585, 578, 586, 579, 587, 583, 590, 591, 592, 1082, 598, 508, 509, 535, 510, 538, 1463, 539, 540, 547, 553, 554, 557, 573, 581, 1390, 892, 1219, 1377, 1234, 599, 605, 606, 607, 608, 609, 611, 610, 612, 601, 600, 603, 1464, 1465, 604, 1220, 1397, 1221, 1223, 613, 629, 1240, 1241, 1242, 1243, 1244, 1250, 1289, 614, 615, 616, 617, 618, 620, 621, 1327, 630, 1245, 1323, 1328, 631, 1320, 1321, 1322, 634, 1247, 1248, 1249, 1083, 619, 1292, 1294, 1084, 1326, 1286, 1288, 1290, 622, 623, 624, 625, 626, 627, 628, 635, 1291, 1295, 1299, 1316, 1317, 632, 1246, 1252, 1253, 1254, 1255, 633, 1296, 1297, 1298, 1318, 1319, 1324, 1325, 1287, 1293, 1236, 1376, 1455, 1456, 1457, 1458, 1461, 1462, 642, 643, 648, 649, 1182, 650, 959, 651, 652, 1157, 653, 654, 759, 644, 655, 1158, 656, 657, 658, 645, 659, 660, 1227, 647, 664, 1235, 1348, 1378, 1353, 1356, 1355, 1360, 1362, 1365, 1366, 1367, 1368, 1382, 1383, 1369, 1370, 1371, 1372, 661, 646, 662, 663, 799, 1132, 1133, 668, 669, 674, 1229, 672, 673, 676, 677, 695, 697, 698, 699, 700, 670, 678, 679, 680, 683, 701, 702, 703, 704, 681, 1215, 1216, 1217, 1315, 728, 729, 730, 732, 733, 769, 770, 771, 772, 773, 774, 775, 776, 1222, 734, 738, 739, 762, 743, 745, 747, 748, 764, 744, 749, 754, 755, 757, 750, 752, 753]
names = [91, 111, 106, 104, 101, 327, 331, 332, 335, 94, 346, 348, 352, 401, 798, 92, 245, 325, 97, 1198, 1124, 1126, 1205, 1206, 1183, 1204, 1186, 1187, 1188, 1190, 1191, 90, 147, 159, 161, 1080, 157, 1081, 1100, 1101, 1102, 1103, 150, 173, 174, 175, 443, 1099, 154, 176, 177, 178, 179, 180, 156, 158, 162, 163, 1104, 1237, 164, 165, 166, 167, 916, 919, 168, 169, 170, 171, 172, 917, 918, 1105, 89, 141, 142, 143, 139, 136, 137, 138, 133, 135, 134, 131, 126, 130, 129, 127, 128, 123, 125, 124, 116, 122, 119, 121, 120, 117, 118, 936, 105, 114, 115, 113, 112, 110, 108, 107, 937, 103, 93, 98, 102, 99, 100, 96, 95, 132, 144, 145, 146, 148, 149, 151, 152, 153, 457, 458, 637, 638, 459, 460, 461, 639, 640, 462, 464, 641, 466, 595, 596, 597, 636, 181, 182, 193, 200, 446, 197, 218, 219, 220, 221, 222, 223, 224, 225, 198, 201, 235, 202, 240, 243, 448, 449, 451, 452, 203, 204, 445, 970, 226, 228, 229, 230, 231, 227, 199, 249, 250, 263, 274, 276, 277, 1388, 1422, 292, 1389, 303, 306, 307, 308, 1384, 1385, 1386, 1387, 310, 312, 314, 316, 1424, 1145, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 456, 419, 420, 421, 422, 838, 839, 840, 841, 842, 843, 884, 423, 424, 425, 426, 427, 1497, 428, 429, 430, 997, 1028, 1029, 1030, 1031, 1046, 1144, 1121, 1127, 1261, 1262, 1334, 1430, 431, 1189, 432, 433, 435, 436, 437, 439, 442, 453, 454, 1079, 705, 194, 196, 210, 1499, 1500, 1501, 706, 707, 708, 709, 711, 725, 712, 713, 714, 715, 716, 717, 767, 768, 1143, 1214, 1345, 720, 721, 722, 1213, 723, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 492, 493, 494, 495, 496, 497, 498, 499, 500, 501, 502, 503, 504, 505, 506, 507, 560, 563, 564, 566, 570, 589, 1074, 576, 585, 578, 586, 579, 587, 583, 590, 591, 592, 1082, 598, 508, 509, 535, 510, 538, 1463, 539, 540, 547, 553, 554, 557, 573, 581, 1390, 892, 1219, 1377, 1234, 601, 600, 603, 1464, 1465, 604, 1220, 1397, 1221, 1223, 613, 629, 1240, 1241, 1242, 1243, 1244, 1250, 1289, 614, 615, 616, 617, 618, 620, 621, 1327, 630, 1245, 1323, 1328, 631, 1320, 1321, 1322, 634, 1247, 1248, 1249, 1083, 619, 1292, 1294, 1084, 1326, 1286, 1288, 1290, 622, 623, 624, 625, 626, 627, 628, 635, 1291, 1295, 1299, 1316, 1317, 632, 1246, 1252, 1253, 1254, 1255, 633, 1296, 1297, 1298, 1318, 1319, 1324, 1325, 1287, 1293, 1236, 1376, 1461, 1462, 642, 643, 648, 649, 1182, 650, 959, 651, 652, 1157, 653, 654, 759, 644, 655, 1158, 656, 657, 658, 645, 659, 660, 1227, 647, 664, 1235, 1348, 1378, 1353, 1356, 1355, 1360, 1362, 1365, 1366, 1367, 1368, 1382, 1383, 1369, 1370, 1371, 1372, 661, 646, 662, 663, 799, 1132, 1133, 668, 669, 674, 1229, 672, 673, 676, 677, 695, 697, 698, 699, 700, 670, 678, 679, 680, 683, 701, 702, 703, 704, 681, 1215, 1216, 1217, 1315, 728, 729, 730, 732, 733, 769, 770, 771, 772, 773, 774, 775, 776, 1222, 734, 738, 739, 762, 743, 745, 747, 748, 764, 744, 749, 754, 755, 757, 750, 752, 753]

In [4]:
tree = np.load('data/tree.npy')

In [5]:
def dfs(tree, st):
    nums = [tree['id']]
    text = [tree['text']]
    print(st + tree['a_attr']['title'])
    st += '    '
    if tree['children'] != None:
        for i in tree['children']:
            t = dfs(i, st)
            nums.extend(t[0])
            text.extend(t[1])
    return nums, text

In [6]:
tree = np.load('data/tree.npy')
nums = []
text = []
for i in tree:
    st = ''
    t = dfs(i, st)
    nums.extend(t[0])
    text.extend(t[1])

d = np.vstack((nums, text))
categories = dict(d.T)

91. Компьютеры
    111. Оргтехника (принтеры, сканеры, факсы)
    106. Аксессуары
    104. Информационные носители и внешние устройства чтения
    101. Периферийные устройства
        327. Клавиатуры, мыши, комплекты
        331. Компьютерная акустика(наушники, колонки) и микрофоны
        332. Мониторы
        335. Прочие периферийные устройства
    94. Комплектующие
        346. Жесткие диски и сетевые накопители
        348. Видеокарты
        352. Процессоры
        401. Материнские платы
        798. Прочие комплектующие
    92. ПК
        245. Моноблоки и стационарные компьютеры
        325. Ноутбуки, нетбуки
    97. Сетевое оборудование
        1198. Настройка сетевого оборудования
    1124. Ремонт компьютеров
        1126. Тест веб-камеры
        1205. Советы по настройке и использованию ПК
            1206. Настройка Windows
    1183. Программное обеспечение
        1204. Драйверы
1186. Эмиграция
    1187. Учеба за границей
    1188. Работа за границей
1190. Сценическое, студи

In [7]:
def getData(tree, names, allUrls, parent_numb, level, depth):
    dataUrls = []
    dataText = []
    dataCategories = []
    
    number = int(tree['a_attr']['title'].split('.')[0])
    parent = number
    if level < depth:
        parent = parent_numb
        
    
    flag = True
    if len(tree['children']) != 0:
        for i in tree['children']:
            a, b, c = getData(i, names, allUrls, parent, level, depth+1)
            if level - 2 == depth and len(c) != 0:
                dataUrls.append(a)
                dataText.append(b)
                dataCategories.append(c)
            else:
                dataUrls.extend(a)
                dataText.extend(b)
                dataCategories.extend(c)
                
    elif level - 1 == depth:
        flag = False
#     else:
#         if number in names and (len(categUrls) - len(dataUrls)) / len(categUrls) > 0.8:
#             dataCategories = (np.ones(len(dataCategories)) * parent_numb).tolist()
    
    if number in names and level - 1 <=  depth and flag and number in files:
        f = open('data/filterData/categories/{}.txt'.format(number))
        lines = f.readlines()
        categUrls = [i.split(', ', 1)[0] for i in lines]
        categText = [i.split(', ', 1)[1] for i in lines]
        f.close()
        
        for i in range(len(categUrls)):
            if categUrls[i] not in allUrls:
                dataUrls.extend([categUrls[i]])
                dataText.append([categText[i].replace('\n', '')])
                dataCategories.extend([parent])

                allUrls.add(categUrls[i])
    
    return dataUrls, dataText, dataCategories

In [8]:
files = glob.glob('data/filterData/categories/*')
files = [int(i[27:].split('.')[0]) for i in files]

In [9]:
allLevels = {}
allUrls = set()
dataUrls = []
dataText = []
dataCategories = []
parent_numb = 1
level = 0
for i in tree:
    a, b, c = getData(i, names, allUrls, parent_numb, level, 0)
    if level == 1:
        dataUrls.append(a)
        dataText.append(b)
        dataCategories.append(c)
    else:
        dataUrls.extend(a)
        dataText.extend(b)
        dataCategories.extend(c)

allLevels[0] = (dataUrls, dataText, dataCategories)

In [10]:
testAns = np.load('data/filterData/TestAns.npy')
testUrl = [i[0] for i in testAns]

In [11]:
testText = []
testCategories = []
for i in testUrl:
    txt = ''
    cat = 0
    for j in allLevels:
        if i in allLevels[j][0]:
            idx = allLevels[j][0].index(i)
            txt = allLevels[j][1][idx]
            cat = allLevels[j][2][idx]
            
            del allLevels[j][0][idx]
            del allLevels[j][1][idx]
            del allLevels[j][2][idx]
    
    testText.extend([txt])
    testCategories.extend([cat])

In [12]:
features = {}
_time = time.time()
for k in allLevels:
    texts = allLevels[k][1]
    
    texts = [i[0] for i in texts]
    
    tfidfTransformer = TfidfVectorizer()
    tfidf = tfidfTransformer.fit_transform(texts)
    
    features[k] = (tfidf, tfidfTransformer, allLevels[k][2])

print(time.time() - _time)

19.090348720550537


# CLF 

In [13]:
X, tfidfTransformer, y = features[0]
testX = tfidfTransformer.transform([i[0] for i in testText])

In [14]:
_time = time.time()
clf = LogisticRegression()
clf.fit(X, y)
print('Fit time: ', time.time() - _time)
predict = clf.predict(testX)

np.save('data/filterData/multilevel/predictLR.npy', predict)

Fit time:  46.18107295036316


In [15]:
_time = time.time()
clf = RidgeClassifier()
clf.fit(X, y)
print('Fit time: ', time.time() - _time)
predict = clf.predict(testX)

np.save('data/filterData/multilevel/predictRC.npy', predict)

Fit time:  76.2439501285553


In [16]:
_time = time.time()
clf = NearestCentroid()
clf.fit(X, y)
print('Fit time: ', time.time() - _time)
predict = clf.predict(testX)

np.save('data/filterData/multilevel/predictNC.npy', predict)

Fit time:  0.9124755859375


In [17]:
_time = time.time()
clf = KNeighborsClassifier()
clf.fit(X, y)
print('Fit time: ', time.time() - _time)
predict = clf.predict(testX)

np.save('data/filterData/multilevel/predictKNN1.npy', predict)

Fit time:  0.09377861022949219


In [ ]:
_time = time.time()
clf = KNeighborsClassifier(algorithm='brute', metric='cosine')
clf.fit(X, y)
print('Fit time: ', time.time() - _time)
predict = clf.predict(testX)

np.save('data/filterData/multilevel/predictKNN2.npy', predict)

Fit time:  3.9524567127227783


In [14]:
_time = time.time()
clf = SVC(gamma=0.5)
clf.fit(X, y)
print('Fit time: ', time.time() - _time)
predict = clf.predict(testX)

np.save('data/filterData/multilevel/predictSVC1.npy', predict)

Fit time:  1580.0233235359192


In [13]:
predict = np.load('data/filterData/multilevel/predictLR.npy')
print(accuracy_score(testCategories, predict))

0.855087843415


In [14]:
predict = np.load('data/filterData/multilevel/predictRC.npy')
print(accuracy_score(testCategories, predict))

0.871477420116


In [15]:
predict = np.load('data/filterData/multilevel/predictNC.npy')
print(accuracy_score(testCategories, predict))

0.787878787879


In [16]:
predict = np.load('data/filterData/multilevel/predictKNN1.npy')
print(accuracy_score(testCategories, predict))

0.457729041387


In [17]:
predict = np.load('data/filterData/multilevel/predictKNN2.npy')
print(accuracy_score(testCategories, predict))

0.819360924419


In [18]:
predict = np.load('data/filterData/multilevel/predictSVC1.npy')
print(accuracy_score(testCategories, predict))

0.855087843415


# Matrix

In [21]:
predict = np.load('data/filterData/multilevel/predictRC.npy')

In [22]:
categories = np.load('data/categor.npy').item()

In [23]:
# labels = list(set(trainCategory) | set(testCategory))
labels = names
matr = confusion_matrix(testCategories, predict, labels=labels)
for i in range(len(labels)):
#     print('Label:', labels[i], '\tAll:', matr[i].sum(), ' \tTrue:', matr[i, i], '\tMax:',\
#           labels[matr[i].argmax()], '\tMaxS:', matr[i].max(), '\tProc:', matr[i, i]/matr[i].sum())
#     if str(int(labels[i])) in categories:
    if matr[i].sum() != 0:
        srt = matr[i].argsort()[::-1]
        print(categories[str(int(labels[i]))])
        print('All:', matr[i].sum(), ' \tPredict:', matr[i, i], '\t%:', matr[i, i] / matr[i].sum())
        for j in range(3):
            if srt[j] != i:
                print('Predict:', matr[i, srt[j]], '\t', categories[str(int(labels[srt[j]]))])
        print()

91. Компьютеры
All: 199  	Predict: 158 	%: 0.793969849246
Predict: 20 	 431. Мобильные устройства
Predict: 6 	 249. Бытовая техника

1186. Эмиграция
All: 86  	Predict: 71 	%: 0.825581395349
Predict: 8 	 728. Туризм по странам
Predict: 2 	 405. Досуг

1190. Сценическое, студийное и профессиональное музыкальное оборудование
All: 87  	Predict: 77 	%: 0.885057471264
Predict: 4 	 405. Досуг
Predict: 3 	 598. Авто и мото

90. Всё для спорта и туризма
All: 788  	Predict: 662 	%: 0.840101522843
Predict: 43 	 405. Досуг
Predict: 25 	 598. Авто и мото

89. Игровые интересы
All: 475  	Predict: 405 	%: 0.852631578947
Predict: 23 	 405. Досуг
Predict: 17 	 431. Мобильные устройства

457. Недвижимость
All: 487  	Predict: 432 	%: 0.887063655031
Predict: 11 	 598. Авто и мото
Predict: 10 	 405. Досуг

181. Ремонт
All: 850  	Predict: 742 	%: 0.872941176471
Predict: 60 	 705. Строительство
Predict: 15 	 405. Досуг

249. Бытовая техника
All: 176  	Predict: 134 	%: 0.761363636364
Predict: 18 	 705. Строит